In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier, StackingClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import plot_roc_curve
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

### deal with some left over from preprocessing

In [3]:
df = pd.read_csv('step2_data/combined.csv', index_col=0)

C:\Users\Alan\AppData\Local\Temp\ipykernel_43868\4094612143.py:1: DtypeWarning: Columns (21,47,51,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('step2_data/combined.csv', index_col=0)


In [5]:
df = df[df['RestaurantsPriceRange2'] != 'None']
df['RestaurantsPriceRange2'] = df['RestaurantsPriceRange2'].astype(float)
df = df.dropna(subset='RestaurantsPriceRange2')
df['RestaurantsPriceRange2'] = df['RestaurantsPriceRange2'].astype(int)

In [6]:
# AcceptsInsurance, Open24Hours RestaurantsCounterService, has no useful value
# DietaryRestrictions have only 5 records
df.drop(columns=['AcceptsInsurance', 'Open24Hours', 'DietaryRestrictions', 'RestaurantsCounterService'], inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7002 entries, 0 to 13049
Columns: 677 entries, business_id to Yoga
dtypes: float64(10), int32(1), int64(630), object(36)
memory usage: 36.2+ MB


In [8]:
for i in df.columns:
    if df[i].dtype == object:
        print(i)

business_id
name
city
RestaurantsDelivery
OutdoorSeating
BusinessAcceptsCreditCards
BusinessParking
BikeParking
RestaurantsTakeOut
ByAppointmentOnly
WiFi
Alcohol
Caters
RestaurantsReservations
RestaurantsGoodForGroups
RestaurantsAttire
HasTV
Ambience
GoodForKids
GoodForMeal
NoiseLevel
DogsAllowed
HappyHour
WheelchairAccessible
RestaurantsTableService
Smoking
Music
GoodForDancing
BusinessAcceptsBitcoin
CoatCheck
BestNights
Corkage
BYOBCorkage
BYOB
DriveThru
AgesAllowed


In [9]:
for c in df.columns:
    print(df[c].value_counts())

MTSW4McQd7CbVtyjqoe9mw    1
zujdPV3HT-Y-CKE1GgkMHQ    1
MKsCFQuzmgH2We9hwTQ8gw    1
lqj-4ZoMcNSNgaNfyn4FMg    1
E_TWqkEwtzwhrRtyDIU2nw    1
                         ..
A9bqf3aoiQXX9WBswewivg    1
3FKIev7ZB_KE6XHL9sUJCg    1
_8yGCHO9xX7J6bWRB3oNwA    1
X1KTKksxwFTg_t21iWPMMA    1
8n93L-ilMAsvwUatarykSg    1
Name: business_id, Length: 7002, dtype: int64
McDonald's                45
PrimoHoagies              28
Chipotle Mexican Grill    27
Domino's Pizza            27
Pizza Hut                 24
                          ..
Philadium Tavern           1
Red King's BBQ             1
Easton Buffet              1
Roast+Chop                 1
Tata Cafe                  1
Name: name, Length: 5919, dtype: int64
Philadelphia       3338
West Chester        158
Doylestown           90
Phoenixville         89
Wayne                88
                   ... 
Wrightstown           1
Philadephia           1
Eddystone             1
Bethel Township       1
UPPER MORELAND        1
Name: city, Length: 239,

In [15]:
## True/False issue
true_false_col = ['BusinessAcceptsCreditCards', 'RestaurantsTakeOut', 'RestaurantsReservations', 'RestaurantsGoodForGroups']

def tf_issue(column_name, df):

    for i in df[column_name]:

        if df[column_name][i] == 'True': df[column_name][i]=1
        elif df[column_name][i] == 'False': df[column_name][i]=0
        else: df[column_name][i]=0
        
    # df = df[df[column_name] != 'None']
    # df[column_name] = df[column_name].astype(float)
    # df = df.dropna(subset=column_name)
    # df[column_name] = df[column_name].astype(int)

for i in true_false_col:
    tf_issue(i, df)

# dic: BusinessParking, GoodForMeal
# 
# weirdo u
# WiFi, Alcohol, RestaurantsAttire, NoiseLevel, Smoking,

KeyError: False

In [17]:
df['BusinessAcceptsCreditCards'].dtype

dtype('O')

In [23]:
df['BusinessAcceptsCreditCards'].value_counts()

True     5617
True      942
False     367
False      61
None        1
Name: BusinessAcceptsCreditCards, dtype: int64

In [13]:
str = 'True'
if 'True' == str:
    print(1)

1


In [ ]:
drop_col = ['Caters', 'HasTV', 'GoodForKids', 'DogsAllowed', 'HappyHour', 'WheelchairAccessible']

In [18]:
X = df.drop(columns=['business_id', 'is_open'])
y = df['is_open']

le = LabelEncoder()
X['name'] = le.fit_transform(X['name'])
X['city'] = le.fit_transform(X['city'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

In [ ]:
scaler = preprocessing.MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = xgb.XGBClassifier()